Checking the accuracy with the help of Random Forest and XGBOOST Algorithms

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
new_df = df.sample(30000)

In [5]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [6]:
new_df.duplicated().sum()

0

In [7]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
126708,Did anyone filibuster the U.S. Patriot Act?,What is the U.S. PATRIOT Act?
144687,Have we forgot to celebrate other cricketers w...,Are there other Indian cricketers apart from A...
91593,Why is LDL more easily oxidized/damaged than HDL?,Why is HI more acidic than HCl?
136483,Where is PIP?,Foreign Exchange Market: What is a pip?
240058,What piano chords go together?,Which is harder to play in a song on piano: ar...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# text merge
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [9]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [11]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
126708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144687,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91593,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
136483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
240058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90193,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
361383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
temp_df['is_duplicate'] = new_df['is_duplicate']
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
126708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144687,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91593,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
136483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
240058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Split arrays or matrices into random train and test subsets.
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.733

In [17]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7215

So we can see that --> without doing any feature engineering we are getting 72 - 73.5 % accuracy.